# Creating Bike and Pedestrian Road Links

For environment see: https://github.com/gboeing/osmnx 

    conda config --prepend channels conda-forge
    
    conda create -n ox --strict-channel-priority osmnx
    
* also include nb_conda, nb_conda_kernels, and ipykernel

In [1]:
import geopandas as gpd
import osmnx
import pandas as pd
import networkx as nx

In [2]:
#update the attributes retrieved from OSM in settings.py
useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 
                    'surface', 'smoothness','cycleway', 'footway', 'sidewalk']
osmnx.utils.config(useful_tags_way = useful_tags_path)

utn = osmnx.settings.useful_tags_node
oxna = osmnx.settings.osm_xml_node_attrs
oxnt = osmnx.settings.osm_xml_node_tags
utw = osmnx.settings.useful_tags_way
oxwa = osmnx.settings.osm_xml_way_attrs
oxwt = osmnx.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
osmnx.config(all_oneway=True, useful_tags_node=utn, useful_tags_way=utw)

#grab from OSM the bike and pedestrian networks for the MPO
#WARNING: Takes 48 hours to run (for both bike and ped together)
places = ['New Shoreham, Rhode Island','Jamestown, Rhode Island', 'Prudence Island, Rhode Island', 'Rhode Island', 
          'Cape Cod, Massachusetts', 'Chappaquiddick Island, Massachusetts','Monomoy Island, Massachusetts',
          'Long Island, Massachusetts', 'Spectacle Island, Massachusetts', 'Thompson Island, Massachusetts',
          'Little Brewster Island, Massachusetts', 'Great Brewster Island, Massachusetts', 
          'Shag Rocks Island, Massachusetts', 'Calf Island, Massachusetts', 'Little Calf Island, Massachusetts',
          'Middle Brewster Island, Massachusetts', 'Outer Brewster Island, Massachusetts',
          'Moon Island, Massachusetts', 'Rainsford Island, Massachusetts', 'Gallops Island, Massachusetts',
          'Georges Island, Massachusetts', 'Lovells Island, Massachusetts', 'Peddocks Island, Massachusetts',
          'Nantucket, Massachusetts', "Martha's Vineyard, Massachusetts", 'Naushon Island, Massachusetts',
          'Nashawena Island, Massachusetts', 'Gosnold, Massachusetts','Provincetown, Massachusetts',
          'Massachusetts', 'New Hampshire']
count = 0
for place in places:
    try:
        bike_net = osmnx.graph_from_place(place, network_type='bike')
        ped_net = osmnx.graph_from_place(place, network_type='walk')
        if count > 0:
            full_bike_net = nx.compose(full_bike_net, bike_net)
            full_ped_net = nx.compose(full_ped_net, ped_net)
        else:
            full_bike_net = bike_net
            full_ped_net = ped_net
        count+=1
    except:
        print(place)
        


Spectacle Island, Massachusetts
Little Brewster Island, Massachusetts
Great Brewster Island, Massachusetts
Shag Rocks Island, Massachusetts
Calf Island, Massachusetts
Little Calf Island, Massachusetts
Middle Brewster Island, Massachusetts
Outer Brewster Island, Massachusetts
Moon Island, Massachusetts
Rainsford Island, Massachusetts
Gallops Island, Massachusetts
Lovells Island, Massachusetts


In [3]:
osmnx.io.save_graph_shapefile(full_bike_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Bike2')

C:\Users\phoebe\anaconda3\envs\ox\lib\site-packages\osmnx\io.py:108: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(filepath_nodes, driver="ESRI Shapefile", index=True, encoding=encoding)


In [4]:
osmnx.io.save_graph_shapefile(full_ped_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Ped2')

C:\Users\phoebe\anaconda3\envs\ox\lib\site-packages\osmnx\io.py:108: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(filepath_nodes, driver="ESRI Shapefile", index=True, encoding=encoding)
